In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
import numpy as np
import unicodedata
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('../data/train_full_norm.csv',header=None,names=['title'],nrows=2000000)
val = pd.read_csv('../data/val_full_norm.csv',header=None,names=['title'], nrows=500000)
test = pd.read_csv('../data/test_full_norm.csv',header=None,names=['title'])

In [3]:
''.join(train['title'][0].split()[0][9:])

'HABERDASHERY_RIBBONS'

In [12]:
train['category'] = train['title'].apply(lambda x: ''.join(x.split()[0][9:]))
train['title'] = train['title'].apply(lambda x: ' '.join(x.split()[1:]))

In [5]:
classes = train.category.unique()

In [13]:
val['category'] = val['title'].apply(lambda x: ''.join(x.split()[0][9:]))
val['title'] = val['title'].apply(lambda x: ' '.join(x.split()[1:]))

In [14]:
y_train = train['category']
y_val = val['category']

In [15]:
count_vect = CountVectorizer()
X_train_vect = count_vect.fit_transform(train['title'])
X_val_vect = count_vect.transform(val['title'])
X_train_vect.shape

(2000000, 233598)

In [16]:
X_train_vect.shape

(2000000, 233598)

In [18]:
clf = XGBClassifier()

In [ ]:
clf.fit(X_train_vect, y_train)
yPrediction = clf.predict(X_val_vect)
print("Balanced Accuracy Score: %.2f" % balanced_accuracy_score(y_val, yPrediction))

In [26]:
yPrediction = clf.predict(X_val_vect)
print("Balanced Accuracy Score: %.2f" % balanced_accuracy_score(y_val, yPrediction))

Balanced Accuracy Score: 0.50


In [ ]:
%time yTrainPredict = clf.predict(X_train_vect)
%time yPrediction = clf.predict(X_test_vect)


In [ ]:
print("Balanced Accuracy Score: %.2f" % balanced_accuracy_score(y_train, yTrainPredict))
print("Balanced Accuracy Score: %.2f" % balanced_accuracy_score(y_val, yPrediction))

## DASK!

In [38]:
import dask
import dask.dataframe as dd 
import dask.array as da
from dask_ml.wrappers import Incremental
from dask.distributed import Client


In [32]:
client = Client(n_workers=8, threads_per_worker=1)
client

/home/franco_camporeale/miniconda3/envs/meli/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:35185 Dashboard: http://127.0.0.1:34899/status,Cluster Workers: 8 Cores: 8 Memory: 31.62 GB


In [36]:
train = dd.read_csv('../data/train_fasttext_reliable_norm.csv',header=None,names=['title'])
val = dd.read_csv('../data/val_fasttext_reliable_norm.csv',header=None,names=['title'])
test = dd.read_csv('../data/test_fasttext.csv',header=None,names=['title'])

In [39]:
train, val, test  = dask.persist(train, val, test)

In [ ]:
classes = da.unique(y_train).compute()

In [40]:
train['category'] = train['title'].apply(lambda x: ''.join(x.split()[0][9:]))
train['title'] = train['title'].apply(lambda x: ' '.join(x.split()[1:]))

/home/franco_camporeale/miniconda3/envs/meli/lib/python3.7/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'object'))

  warnings.warn(meta_warning(meta))


In [48]:
count_vect = CountVectorizer()
X_train_vect = count_vect.fit_transform(train['title'] )
X_val_vect = count_vect.transform(val['title'])
X_train_vect.shape

(2250065, 234927)

In [41]:
val['category'] = val['title'].apply(lambda x: ''.join(x.split()[0][9:]))
val['title'] = val['title'].apply(lambda x: ' '.join(x.split()[1:]))

In [42]:
y_train = train['category']
y_val = val['category']

In [52]:
type(X_train_vect)

scipy.sparse.csr.csr_matrix

In [44]:
classes = da.unique(y_train.values).compute()

In [46]:
len(classes)

1588

In [27]:
mnb = MultinomialNB()

In [28]:
mnb_inc = Incremental(mnb, scoring='balanced_accuracy')

In [53]:
X_train_vect

<2250065x234927 sparse matrix of type '<class 'numpy.int64'>'
	with 13688662 stored elements in Compressed Sparse Row format>

In [50]:
mnb_inc.fit(X_train_vect, y_train, classes=classes)

AttributeError: chunks not found